In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy 
import random
import shutil, sys
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

#print("PyTorch Version: ",torch.__version__)
#print("Torchvision Version: ",torchvision.__version__)

In [ ]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = '/home/jimi/dissertacao/covid19/datasets/dataset_bal/'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
#model_name = "densenet"

# Number of classes in the dataset
#num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for


# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

#num_epochs = 500

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
#transforms.RandomRotation(degrees=(-5, 5)),
#transforms.ColorJitter(brightness=.02),
    
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(degrees=(-5, 5)),
        transforms.RandomAffine(degrees=(0), translate=(0.0, 0.1), scale=None, shear=None, resample=False, fillcolor=None),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.RandomRotation(degrees=(-5, 5)),
        transforms.RandomAffine(degrees=(0), translate=(0.0, 0.1), scale=None, shear=None, resample=False, fillcolor=None),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x]) for x in ['train', 'val']}

# Create training and validation dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                    batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Path to data
data_dir = '/home/jimi/dissertacao/covid19/datasets/dataset_bal/'
train_dir = data_dir+'train/'
test_dir = data_dir+'test/'
val_dir = data_dir+'val/'

normal_dir = data_dir+'normal/'
pneumonia_dir = data_dir+'pneumonia/'
covid_dir = data_dir+'covid/'

len_covid = len([iq for iq in os.scandir(normal_dir)])
len_normal = len([iq for iq in os.scandir(pneumonia_dir)])
len_pneumonia = len([iq for iq in os.scandir(covid_dir)])

len_train_dir = len([iq for iq in os.scandir(train_dir+'covid/')]) + len([iq for iq in os.scandir(train_dir+'normal/')]) + len([iq for iq in os.scandir(train_dir+'pneumonia/')])
len_test_dir = len([iq for iq in os.scandir(test_dir+'covid/')]) + len([iq for iq in os.scandir(test_dir+'normal/')]) + len([iq for iq in os.scandir(test_dir+'pneumonia/')])
len_val_dir = len([iq for iq in os.scandir(val_dir+'covid/')]) + len([iq for iq in os.scandir(val_dir+'normal/')]) + len([iq for iq in os.scandir(val_dir+'pneumonia/')])

print('Files per classes')
print("----"*10)
print("normal : ", len_covid)
print("pneumonia : ", len_normal)
print("covid : ", len_pneumonia)
print("-"*20)
print('Train, test, validation')
print("-"*20)
print("len_train_dir : ", len_train_dir)
print("len_test_dir : ", len_test_dir)
print("len_val_dir : ", len_val_dir)


In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)


imshow(out, title=[class_names[x] for x in classes])

In [ ]:
for i in range (8):
    plt.subplot(2,4,i+1)
    plt.imshow(inputs[i][0], cmap='gray')

# Make a grid from batch
img_grid = torchvision.utils.make_grid(inputs)

#tensorboard
#writer.add_image('Chest X Ray', img_grid)

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs, model_name, lr, batch_size):
    since = time.time()
    is_inception = False
    
    #tensorboard
    writer = SummaryWriter(f'runs/dg_{model_name}_lr={lr}_epoch={num_epochs}_batch_size={batch_size}')
    step = 0

    val_acc_history = []
    val_loss_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            writer.add_scalar('training loss', loss, global_step=step)
            writer.add_scalar('training accuracy', epoch_acc, global_step=step)
            step += 1
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()
        
        
    time_elapsed = time.time() - since
    print()
    print('#'*30)
    print('------ Summary ------')
    print(f'model -> {_model}')
    print(f'epochs -> {_epochs}')
    print(f'lr -> {_lrs}')
    print(f'batch size -> {_batch}')
    print()
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('#'*30)
    print()
    print('==== END ====')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def visualize_model(model, num_images=4):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 3, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


In [ ]:
# #parmeters
# num_epochs = num_epochs
# lr=0.001

# num_epochs = 500
# lr=0.0001

# num_epochs = 500
# lr=0.0001

# num_epochs = 500
# lr=0.001

# num_epochs = 500
# lr=0.00002

# Trainamento

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
num_classes = 3

_models = ['squeezenet', 'densenet', 'resnet', 'alexnet', 'vgg']
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
_epoch = [30, 50, 100, 200, 500]
batch_sizes = [8, 16, 32]

for _model in _models:
    for _epochs in _epoch:
        for _lrs in lrs:
            for _batch in batch_sizes:
                               
                print()
                print()
                print()
                print('='*60)
                print('==== INITIALIZING WITH PARAMETERS: ====')
                print()
                print(f'model -> {_model}')
                print(f'epochs -> {_epochs}')
                print(f'lr -> {_lrs}')
                print(f'batch size -> {_batch}')
                print()

                feature_extract = True
                
                model_ft, input_size = initialize_model(_model, num_classes, 
                                                        feature_extract, use_pretrained=True)
                
                # Send the model to GPU
                model_ft = model_ft.to(device)

                print('-'*20)
                params_to_update = model_ft.parameters()
                print("Params to learn:")
                if feature_extract:
                    params_to_update = []
                    for name,param in model_ft.named_parameters():
                        if param.requires_grad == True:
                            params_to_update.append(param)
                            print("\t",name)
                else:
                    for name,param in model_ft.named_parameters():
                        if param.requires_grad == True:
                            print("\t",name)

                print()
                print('-'*20)
                print()
                print('== Epochs ==')
                
                optimizer_ft = optim.SGD(params_to_update, _lrs, momentum=0.9)
                exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

                criterion = nn.CrossEntropyLoss()

                model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                                             num_epochs=_epochs, model_name=_model, lr=_lrs, batch_size=_batch)

val Loss: 0.7904 Acc: 0.5761

Epoch 9/29
----------
train Loss: 0.7603 Acc: 0.5635
val Loss: 0.7638 Acc: 0.6008

Epoch 10/29
----------
train Loss: 0.7447 Acc: 0.5831
val Loss: 0.7476 Acc: 0.6255

Epoch 11/29
----------
train Loss: 0.7507 Acc: 0.5697
val Loss: 0.7085 Acc: 0.6091

Epoch 12/29
----------
train Loss: 0.7490 Acc: 0.5666
val Loss: 0.7530 Acc: 0.5802

Epoch 13/29
----------
train Loss: 0.7292 Acc: 0.5872
val Loss: 0.7635 Acc: 0.6049

Epoch 14/29
----------
train Loss: 0.7268 Acc: 0.5831
val Loss: 0.7837 Acc: 0.5802

Epoch 15/29
----------
train Loss: 0.7169 Acc: 0.5872
val Loss: 0.8031 Acc: 0.6008

Epoch 16/29
----------
train Loss: 0.7123 Acc: 0.5862
val Loss: 0.8030 Acc: 0.5885

Epoch 17/29
----------
train Loss: 0.7311 Acc: 0.5800
val Loss: 0.7837 Acc: 0.5802

Epoch 18/29
----------
train Loss: 0.7326 Acc: 0.5748
val Loss: 0.7601 Acc: 0.5885

Epoch 19/29
----------
train Loss: 0.7347 Acc: 0.5851
val Loss: 0.7215 Acc: 0.6132

Epoch 20/29
----------
train Loss: 0.7381 Acc: 

train Loss: 0.8362 Acc: 0.5728
val Loss: 0.3710 Acc: 0.8724

Epoch 1/29
----------
train Loss: 0.5272 Acc: 0.7843
val Loss: 1.0164 Acc: 0.6626

Epoch 2/29
----------
train Loss: 0.4798 Acc: 0.8142
val Loss: 0.3297 Acc: 0.9053

Epoch 3/29
----------
train Loss: 0.4128 Acc: 0.8390
val Loss: 0.2670 Acc: 0.9012

Epoch 4/29
----------
train Loss: 0.3924 Acc: 0.8493
val Loss: 0.3281 Acc: 0.8889

Epoch 5/29
----------
train Loss: 0.3328 Acc: 0.8700
val Loss: 0.4768 Acc: 0.7942

Epoch 6/29
----------
train Loss: 0.3523 Acc: 0.8741
val Loss: 0.2646 Acc: 0.9300

Epoch 7/29
----------
train Loss: 0.3484 Acc: 0.8638
val Loss: 0.2857 Acc: 0.9095

Epoch 8/29
----------
train Loss: 0.3462 Acc: 0.8710
val Loss: 0.2451 Acc: 0.9095

Epoch 9/29
----------
train Loss: 0.3284 Acc: 0.8762
val Loss: 0.2565 Acc: 0.9342

Epoch 10/29
----------
train Loss: 0.2774 Acc: 0.8968
val Loss: 0.2720 Acc: 0.9095

Epoch 11/29
----------
train Loss: 0.3609 Acc: 0.8669
val Loss: 0.3111 Acc: 0.9136

Epoch 12/29
----------
t

train Loss: 0.2877 Acc: 0.8989
val Loss: 0.1892 Acc: 0.9547

Epoch 27/29
----------
train Loss: 0.2949 Acc: 0.8937
val Loss: 0.2155 Acc: 0.9465

Epoch 28/29
----------
train Loss: 0.2817 Acc: 0.8958
val Loss: 0.2139 Acc: 0.9383

Epoch 29/29
----------
train Loss: 0.3010 Acc: 0.8854
val Loss: 0.2504 Acc: 0.9300


##############################
------ Summary ------
model -> squeezenet
epochs -> 30
lr -> 0.0001
batch size -> 8

Training complete in 5m 13s
Best val Acc: 0.954733
##############################

==== FIM DO TESTE ====



==== INICIANDO UM TESTE COM OS PARAMETROS: ====

model -> squeezenet
epochs -> 30
lr -> 0.0001
batch size -> 16

--------------------
Params to learn:
	 classifier.1.weight
	 classifier.1.bias

--------------------

== Epochs ==
Epoch 0/29
----------
train Loss: 0.9297 Acc: 0.5366
val Loss: 0.6554 Acc: 0.7860

Epoch 1/29
----------
train Loss: 0.5936 Acc: 0.7719
val Loss: 0.4446 Acc: 0.8765

Epoch 2/29
----------
train Loss: 0.5019 Acc: 0.8194
val Loss: 0.4

train Loss: 0.5190 Acc: 0.8163
val Loss: 0.4241 Acc: 0.8724

Epoch 17/29
----------
train Loss: 0.5277 Acc: 0.7957
val Loss: 0.4253 Acc: 0.8477

Epoch 18/29
----------
train Loss: 0.4763 Acc: 0.8297
val Loss: 0.4035 Acc: 0.8724

Epoch 19/29
----------
train Loss: 0.4953 Acc: 0.8349
val Loss: 0.4016 Acc: 0.8683

Epoch 20/29
----------
train Loss: 0.5066 Acc: 0.7998
val Loss: 0.3801 Acc: 0.8724

Epoch 21/29
----------
train Loss: 0.4851 Acc: 0.8266
val Loss: 0.4052 Acc: 0.8848

Epoch 22/29
----------
train Loss: 0.4625 Acc: 0.8411
val Loss: 0.3639 Acc: 0.8848

Epoch 23/29
----------
train Loss: 0.4609 Acc: 0.8400
val Loss: 0.3767 Acc: 0.8889

Epoch 24/29
----------
train Loss: 0.4516 Acc: 0.8277
val Loss: 0.3729 Acc: 0.8765

Epoch 25/29
----------
train Loss: 0.4514 Acc: 0.8256
val Loss: 0.3686 Acc: 0.8765

Epoch 26/29
----------
train Loss: 0.4540 Acc: 0.8369
val Loss: 0.3558 Acc: 0.9136

Epoch 27/29
----------
train Loss: 0.4424 Acc: 0.8380
val Loss: 0.3423 Acc: 0.9095

Epoch 28/29
---

train Loss: 0.5371 Acc: 0.7967
val Loss: 0.4048 Acc: 0.8889

Epoch 7/49
----------
train Loss: 0.5262 Acc: 0.7946
val Loss: 0.4343 Acc: 0.9012

Epoch 8/49
----------
train Loss: 0.4161 Acc: 0.8431
val Loss: 0.3516 Acc: 0.9259

Epoch 9/49
----------
train Loss: 0.4221 Acc: 0.8369
val Loss: 0.5704 Acc: 0.8272

Epoch 10/49
----------
train Loss: 0.4600 Acc: 0.8194
val Loss: 0.3214 Acc: 0.9259

Epoch 11/49
----------
train Loss: 0.4374 Acc: 0.8369
val Loss: 0.3938 Acc: 0.9012

Epoch 12/49
----------
train Loss: 0.4259 Acc: 0.8514
val Loss: 0.3962 Acc: 0.9012

Epoch 13/49
----------
train Loss: 0.3667 Acc: 0.8576
val Loss: 0.4370 Acc: 0.8560

Epoch 14/49
----------
train Loss: 0.5381 Acc: 0.7781
val Loss: 0.4949 Acc: 0.8436

Epoch 15/49
----------
train Loss: 0.3948 Acc: 0.8596
val Loss: 0.3568 Acc: 0.8971

Epoch 16/49
----------
train Loss: 0.3677 Acc: 0.8596
val Loss: 0.2621 Acc: 0.9259

Epoch 17/49
----------
train Loss: 0.3603 Acc: 0.8731
val Loss: 0.3064 Acc: 0.9053

Epoch 18/49
------

...

train Loss: 0.1970 Acc: 0.9257
val Loss: 0.1869 Acc: 0.9465

Epoch 180/199
----------
train Loss: 0.2409 Acc: 0.9143
val Loss: 0.1752 Acc: 0.9300

Epoch 181/199
----------
train Loss: 0.2445 Acc: 0.9071
val Loss: 0.1830 Acc: 0.9342

Epoch 182/199
----------
train Loss: 0.2274 Acc: 0.9102
val Loss: 0.2261 Acc: 0.9300

Epoch 183/199
----------
train Loss: 0.2194 Acc: 0.9154
val Loss: 0.2554 Acc: 0.9259

Epoch 184/199
----------
train Loss: 0.1908 Acc: 0.9319
val Loss: 0.2496 Acc: 0.9424

Epoch 185/199
----------
train Loss: 0.2404 Acc: 0.9143
val Loss: 0.2057 Acc: 0.9383

Epoch 186/199
----------
train Loss: 0.2255 Acc: 0.9195
val Loss: 0.3621 Acc: 0.8930

Epoch 187/199
----------
train Loss: 0.2409 Acc: 0.9174
val Loss: 0.2312 Acc: 0.9177

Epoch 188/199
----------
train Loss: 0.2254 Acc: 0.9236
val Loss: 0.2142 Acc: 0.9259

Epoch 189/199
----------
train Loss: 0.2437 Acc: 0.9133
val Loss: 0.1742 Acc: 0.9383

Epoch 190/199
----------
train Loss: 0.2359 Acc: 0.9133
val Loss: 0.3081 Acc: 0

val Loss: 0.2541 Acc: 0.9342

Epoch 70/199
----------
train Loss: 0.2394 Acc: 0.9092
val Loss: 0.3708 Acc: 0.9095

Epoch 71/199
----------
train Loss: 0.2416 Acc: 0.8978
val Loss: 0.1877 Acc: 0.9342

Epoch 72/199
----------
train Loss: 0.2609 Acc: 0.8999
val Loss: 0.4110 Acc: 0.8765

Epoch 73/199
----------
train Loss: 0.2582 Acc: 0.8968
val Loss: 0.2660 Acc: 0.9300

Epoch 74/199
----------
train Loss: 0.2746 Acc: 0.8978
val Loss: 0.2164 Acc: 0.9300

Epoch 75/199
----------
train Loss: 0.2599 Acc: 0.8968
val Loss: 0.2493 Acc: 0.9342

Epoch 76/199
----------
train Loss: 0.2378 Acc: 0.9071
val Loss: 0.4065 Acc: 0.8807

Epoch 77/199
----------
train Loss: 0.2480 Acc: 0.9195
val Loss: 0.3129 Acc: 0.9136

Epoch 78/199
----------
train Loss: 0.2350 Acc: 0.9185
val Loss: 0.2111 Acc: 0.9465

Epoch 79/199
----------
train Loss: 0.2788 Acc: 0.9030
val Loss: 0.1888 Acc: 0.9424

Epoch 80/199
----------
train Loss: 0.2411 Acc: 0.9020
val Loss: 0.2220 Acc: 0.9465

Epoch 81/199
----------
train Loss:

train Loss: 0.2438 Acc: 0.9112
val Loss: 0.2723 Acc: 0.9218

Epoch 166/199
----------
train Loss: 0.2211 Acc: 0.9185
val Loss: 0.2393 Acc: 0.9383

Epoch 167/199
----------
train Loss: 0.2563 Acc: 0.9040
val Loss: 0.2356 Acc: 0.9177

Epoch 168/199
----------
train Loss: 0.2350 Acc: 0.9102
val Loss: 0.2713 Acc: 0.9095

Epoch 169/199
----------
train Loss: 0.2129 Acc: 0.9236
val Loss: 0.2080 Acc: 0.9424

Epoch 170/199
----------
train Loss: 0.2293 Acc: 0.9030
val Loss: 0.2166 Acc: 0.9342

Epoch 171/199
----------
train Loss: 0.2280 Acc: 0.9154
val Loss: 0.1915 Acc: 0.9465

Epoch 172/199
----------
train Loss: 0.2368 Acc: 0.9102
val Loss: 0.2515 Acc: 0.9342

Epoch 173/199
----------
train Loss: 0.2594 Acc: 0.9051
val Loss: 0.2238 Acc: 0.9300

Epoch 174/199
----------
train Loss: 0.2275 Acc: 0.9154
val Loss: 0.2580 Acc: 0.9095

Epoch 175/199
----------
train Loss: 0.2124 Acc: 0.9247
val Loss: 0.2033 Acc: 0.9424

Epoch 176/199
----------
train Loss: 0.2473 Acc: 0.9112
val Loss: 0.1909 Acc: 0

val Loss: 0.1966 Acc: 0.9424

Epoch 56/199
----------
train Loss: 0.2483 Acc: 0.9030
val Loss: 0.3694 Acc: 0.8889

Epoch 57/199
----------
train Loss: 0.2635 Acc: 0.9051
val Loss: 0.2225 Acc: 0.9342

Epoch 58/199
----------
train Loss: 0.2384 Acc: 0.9071
val Loss: 0.2783 Acc: 0.9259

Epoch 59/199
----------
train Loss: 0.2411 Acc: 0.9071
val Loss: 0.4047 Acc: 0.8642

Epoch 60/199
----------
train Loss: 0.2572 Acc: 0.9040
val Loss: 0.3141 Acc: 0.9136

Epoch 61/199
----------
train Loss: 0.2684 Acc: 0.8958
val Loss: 0.2092 Acc: 0.9424

Epoch 62/199
----------
train Loss: 0.2808 Acc: 0.8958
val Loss: 0.1715 Acc: 0.9424

Epoch 63/199
----------
train Loss: 0.3215 Acc: 0.8782
val Loss: 0.3033 Acc: 0.9053

Epoch 64/199
----------
train Loss: 0.2551 Acc: 0.9123
val Loss: 0.3048 Acc: 0.9012

Epoch 65/199
----------
train Loss: 0.2230 Acc: 0.9164
val Loss: 0.2036 Acc: 0.9342

Epoch 66/199
----------
train Loss: 0.2382 Acc: 0.9143
val Loss: 0.1889 Acc: 0.9424

Epoch 67/199
----------
train Loss:

val Loss: 0.1628 Acc: 0.9465

Epoch 152/199
----------
train Loss: 0.2358 Acc: 0.9205
val Loss: 0.2929 Acc: 0.9012

Epoch 153/199
----------
train Loss: 0.2498 Acc: 0.9123
val Loss: 0.2365 Acc: 0.9383

Epoch 154/199
----------
train Loss: 0.2215 Acc: 0.9133
val Loss: 0.1952 Acc: 0.9342

Epoch 155/199
----------
train Loss: 0.2294 Acc: 0.9133
val Loss: 0.1666 Acc: 0.9424

Epoch 156/199
----------
train Loss: 0.2552 Acc: 0.9112
val Loss: 0.2008 Acc: 0.9259

Epoch 157/199
----------
train Loss: 0.2065 Acc: 0.9205
val Loss: 0.2210 Acc: 0.9218

Epoch 158/199
----------
train Loss: 0.2361 Acc: 0.9195
val Loss: 0.3185 Acc: 0.9095

Epoch 159/199
----------
train Loss: 0.2070 Acc: 0.9205
val Loss: 0.2021 Acc: 0.9259

Epoch 160/199
----------
train Loss: 0.2290 Acc: 0.9133
val Loss: 0.3405 Acc: 0.9012

Epoch 161/199
----------
train Loss: 0.2094 Acc: 0.9174
val Loss: 0.2247 Acc: 0.9383

Epoch 162/199
----------
train Loss: 0.2329 Acc: 0.9164
val Loss: 0.1996 Acc: 0.9465

Epoch 163/199
----------

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"
#num_classes = 3
#batch_size = 8
#num_epochs = 10000
feature_extract = True
#lr=0.001
#writer.add_graph(model_ft.to(device), inputs)

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

print(model_ft)

optimizer_ft = optim.SGD(params_to_update, lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()

model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                             num_epochs=num_epochs, model_name=model_name, lr=lr)

In [ ]:
visualize_model(model_ft)

In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
print()
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Densenet

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
#num_classes = 3
#batch_size = 8
#num_epochs = 10000
model_name = "densenet"
feature_extract = True
#lr=0.001
#writer.add_graph(model_ft.to(device), inputs)

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

print(model_ft)

optimizer_ft = optim.SGD(params_to_update, lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()


model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                             num_epochs=num_epochs, model_name=model_name, lr=lr)

In [ ]:
visualize_model(model_ft)

In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
print()
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Resnet

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"
#num_classes = 3
#batch_size = 8
#num_epochs = 10000
feature_extract = True
#lr=0.001
#writer.add_graph(model_ft.to(device), inputs)

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

print(model_ft)

optimizer_ft = optim.SGD(params_to_update, lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()


model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                             num_epochs=num_epochs, model_name=model_name, lr=lr)

In [ ]:
visualize_model(model_ft)

In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
print()
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Vgg

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"
#num_classes = 3
#batch_size = 8
#num_epochs = 10000
feature_extract = True
#lr=0.001
#writer.add_graph(model_ft.to(device), inputs)

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

print(model_ft)

optimizer_ft = optim.SGD(params_to_update, lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()


model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                             num_epochs=num_epochs, model_name=model_name, lr=lr)

In [ ]:
visualize_model(model_ft)

In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
print()
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

# Alexnet

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "alexnet"
#num_classes = 3
#batch_size = 8
#num_epochs = 10000
feature_extract = True
#lr=0.001
#writer.add_graph(model_ft.to(device), inputs)

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

print(model_ft)

optimizer_ft = optim.SGD(params_to_update, lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()


model_ft, hist = train_model(model_ft, dataloaders, criterion, optimizer_ft,
                             num_epochs=num_epochs, model_name=model_name, lr=lr)

In [ ]:
visualize_model(model_ft)

In [ ]:
from sklearn.metrics import confusion_matrix

nb_classes = 3

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)
print()
# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)